In [1]:
import pandas as pd
import numpy as np
import regex, re, sys, nltk

from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from gensim.models import Nmf
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter
from collections import Counter

import matplotlib.pyplot as plt

from pathlib import Path

src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)
# python file with all the functions (located in the src folder)
import topic_classification as tc

In [2]:
filepath = Path.cwd().parent / "speeches_csv" / "all_speeches_cleaned.txt"
df=pd.read_csv(filepath, usecols=['title','content'])
df.head()

,title,content
0,CGI_2013,Hillary Clinton: Thank you very much. I have t...
1,Prayer_Breakfast_2016,"Well, good morning. Giving all praise and hono..."
2,Security_Team_Announcement,"Good morning, everybody. I hope you all had a ..."
3,Cairo_University,Thank you so much. Good afternoon. I am honore...
4,Umpqua_Community_College_Shootings,There's been another mass shooting in America ...


In [3]:
df['text']=df['content'].str.lower()
df.head()

,title,content,text
0,CGI_2013,Hillary Clinton: Thank you very much. I have t...,hillary clinton: thank you very much. i have t...
1,Prayer_Breakfast_2016,"Well, good morning. Giving all praise and hono...","well, good morning. giving all praise and hono..."
2,Security_Team_Announcement,"Good morning, everybody. I hope you all had a ...","good morning, everybody. i hope you all had a ..."
3,Cairo_University,Thank you so much. Good afternoon. I am honore...,thank you so much. good afternoon. i am honore...
4,Umpqua_Community_College_Shootings,There's been another mass shooting in America ...,there's been another mass shooting in america ...


In [4]:
# tokenize into sentences
df['tokenized']=df['text'].apply(lambda text: nltk.sent_tokenize(text))
df[['text','tokenized']].head()

,text,tokenized
0,hillary clinton: thank you very much. i have t...,"[hillary clinton: thank you very much., i have..."
1,"well, good morning. giving all praise and hono...","[well, good morning., giving all praise and ho..."
2,"good morning, everybody. i hope you all had a ...","[good morning, everybody., i hope you all had ..."
3,thank you so much. good afternoon. i am honore...,"[thank you so much., good afternoon., i am hon..."
4,there's been another mass shooting in america ...,[there's been another mass shooting in america...


In [5]:
# the '\b(?!\d)' filters out expressions like '9th', since the first character cannot be a number
tokenizer = RegexpTokenizer(r'\b(?!\d)[a-zA-Z]+')
lemmatizer = WordNetLemmatizer()
df['normalized']=df['tokenized'].apply(lambda text: tc.normalize_text(text, tokenizer, lemmatizer))
df[['text','tokenized','normalized']].head()

,text,tokenized,normalized
0,hillary clinton: thank you very much. i have t...,"[hillary clinton: thank you very much., i have...",hillary clinton thank you very much i have th...
1,"well, good morning. giving all praise and hono...","[well, good morning., giving all praise and ho...",well good morning give all praise and honor t...
2,"good morning, everybody. i hope you all had a ...","[good morning, everybody., i hope you all had ...",good morning everybody i hope you all have a ...
3,thank you so much. good afternoon. i am honore...,"[thank you so much., good afternoon., i am hon...",thank you so much good afternoon i be honor t...
4,there's been another mass shooting in america ...,[there's been another mass shooting in america...,there s be another mass shooting in america t...


In [6]:
STOPWORDS = set(stopwords.words('english'))
df['fully_processed'] = df['normalized'].apply(lambda text: tc.remove_stopwords(text, STOPWORDS))
                
cnt = Counter()
for text in df['fully_processed'].values:
    # counts the number of speeches the word is in
    for word in set(text.split()):
        cnt[word] += 1
# words that are in most of the speeches
in_most_speeches = cnt.most_common(155)
in_most_speeches = [x[0] for x in in_most_speeches]

extra = ['mr', 'question', 'sure', 'obama', 'really', 'try', 'lot', 'important', 'million', 'talk', 'va', 'dr', 'romney',
        'folk', 'governor', 'republican', 'king', 'heart'] 

STOPWORDS_extra = set(in_most_speeches + extra)
# remove some words from the stopwords list that migth be important
STOPWORDS_extra = STOPWORDS_extra - set(['war', 'care', 'child', 'family', 'job', 'law', 'protect', 'security', 'power'])

df['fully_processed'] = df['fully_processed'].apply(lambda text: tc.remove_stopwords(text, STOPWORDS_extra))
df[['text','tokenized','normalized','fully_processed']].head()

,text,tokenized,normalized,fully_processed
0,hillary clinton: thank you very much. i have t...,"[hillary clinton: thank you very much., i have...",hillary clinton thank you very much i have th...,hillary clinton pleasure introduce speaker con...
1,"well, good morning. giving all praise and hono...","[well, good morning., giving all praise and ho...",well good morning give all praise and honor t...,morning praise honor god morning everyone orga...
2,"good morning, everybody. i hope you all had a ...","[good morning, everybody., i hope you all had ...",good morning everybody i hope you all have a ...,morning wonderful thanksgiving announce econom...
3,thank you so much. good afternoon. i am honore...,"[thank you so much., good afternoon., i am hon...",thank you so much good afternoon i be honor t...,afternoon honor timeless city cairo host remar...
4,there's been another mass shooting in america ...,[there's been another mass shooting in america...,there s be another mass shooting in america t...,mass shooting college oregon family mom dad ch...


Use gensim's nmf model to find the best number of topics (sklearn doesn't have a way to calculate the coherence score), and use this number of topics with the sklearn nmf model (sklearn can use tf-idf weights, while gensim can only use raw counts)

In [ ]:
# Use Gensim's NMF to get the best num of topics via coherence score
texts = [text.split() for text in df['fully_processed'].values]

# Create a dictionary
# In gensim a dictionary is a mapping between words and their integer id
dictionary = Dictionary(texts)

# Filter out extremes to limit the number of features
dictionary.filter_extremes(
    no_below=3,
    no_above=0.85,
    keep_n=5000
)

# Create the bag-of-words format (list of (token_id, token_count))
corpus = [dictionary.doc2bow(text) for text in texts]

# Create a list of the topic numbers we want to try
topic_nums = list(np.arange(4, 30, 1))

# Run the nmf model and calculate the coherence score
# for each number of topics
coherence_scores = []

for num in topic_nums:
    nmf = Nmf(
        corpus=corpus,
        num_topics=num,
        id2word=dictionary,
        passes=20,
        kappa=.5,
        minimum_probability=0.001,
        w_max_iter=500,
        h_max_iter=100,
        normalize=True,
        random_state=42
    )
    
    # Run the coherence model to get the score
    cm = CoherenceModel(
        model=nmf,
        texts=texts,
        dictionary=dictionary,
        coherence='c_v'
    )
    
    coherence_scores.append(round(cm.get_coherence(), 5))

# Get the number of topics with the highest coherence score
scores = list(zip(topic_nums, coherence_scores))
best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]

print(best_num_topics)

In [ ]:
fig, ax = plt.subplots()
ax.plot(topic_nums, coherence_scores)
ax.set_xlabel('nb of topics')
ax.set_ylabel('coherence score');
ymin = 0.44
ymax = 0.6
plt.vlines(x=best_num_topics, ymin=ymin, ymax=ymax, color='r')
plt.vlines(x=25, ymin=ymin, ymax=ymax, color='k')
plt.vlines(x=12, ymin=ymin, ymax=ymax, color='k')
plt.show()

In [7]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=3, max_df=0.85, ngram_range=(1, 3), max_features=5000)
matrix = vectorizer.fit_transform(df['fully_processed'])
feature_names = vectorizer.get_feature_names_out()
vec_df = pd.DataFrame(matrix.toarray(), columns = feature_names)
vec_df

,abadi,abandon,abe,abide,ability,abraham,abraham lincoln,abroad,absence,absolute,...,yes,yes yes,yesterday,yield,york,york city,youth,zero,zika,zone
0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.004143,0.0,0.000000,0.000000,0.006214,0.0,0.000000,0.0,0.00000,0.0
1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.026362,0.0,...,0.014037,0.0,0.017122,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
2,0.0,0.000000,0.0,0.000000,0.021931,0.000000,0.000000,0.029230,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
3,0.0,0.013339,0.0,0.022761,0.024872,0.015361,0.000000,0.000000,0.000000,0.0,...,0.007367,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
4,0.0,0.000000,0.0,0.060738,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.009882,0.000000,0.0,...,0.000000,0.0,0.008037,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
432,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0
433,0.0,0.013680,0.0,0.000000,0.008502,0.000000,0.000000,0.000000,0.000000,0.0,...,0.060441,0.0,0.000000,0.015753,0.000000,0.0,0.025182,0.0,0.08719,0.0
434,0.0,0.026190,0.0,0.000000,0.000000,0.000000,0.000000,0.021695,0.000000,0.0,...,0.014464,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0


In [15]:
n_topics = 25 # 5 12 25
model = NMF(n_components=n_topics, init='nndsvda', max_iter=500)
model.fit(matrix)

NMF(init='nndsvda', max_iter=500, n_components=25)

In [16]:
n_top_words = 10
#fig, axes = plt.subplots(1, n_topics, figsize=(15, 5), sharex=True)
#axes = axes.flatten()
top_words = []
for topic_idx, topic in enumerate(model.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    print(top_features)
    top_words.append(top_features)
    weights = topic[top_features_ind]

    #ax = axes[topic_idx]
    #ax.barh(top_features, weights, height=0.7)
    #ax.set_title(f'Topic {topic_idx +1}',
    #                 fontdict={'fontsize': 14})
    #ax.invert_yaxis()
    #ax.tick_params(axis='both', which='major', labelsize=10)

['energy', 'oil', 'clean', 'climate', 'clean energy', 'job', 'gas', 'carbon', 'industry', 'economy']
['tax', 'deficit', 'cut', 'job', 'spending', 'pay', 'business', 'economy', 'debt', 'tax cut']
['god', 'faith', 'love', 'child', 'family', 'father', 'grace', 'church', 'prayer', 'pray']
['isil', 'syria', 'terrorist', 'iraq', 'iraqi', 'syrian', 'fighter', 'assad', 'military', 'attack']
['insurance', 'health', 'care', 'health care', 'health insurance', 'coverage', 'affordable', 'insurance company', 'cost', 'affordable care']
['russia', 'ukraine', 'nato', 'russian', 'europe', 'estonia', 'ukrainian', 'separatist', 'international', 'european']
['iran', 'nuclear', 'iranian', 'deal', 'nuclear weapon', 'weapon', 'sanction', 'program', 'nuclear program', 'iran nuclear']
['war', 'peace', 'democracy', 'human', 'freedom', 'africa', 'nuclear', 'europe', 'conflict', 'power']
['gun', 'background check', 'gun violence', 'background', 'check', 'violence', 'gun owner', 'newtown', 'owner', 'law']
['intelli

In [17]:
t_df = pd.DataFrame(model.transform(matrix))
t_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000762,0.032733,0.000000,0.000000,0.406336,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000139,0.000000,0.274539,0.000000,0.000000,0.000000,0.000032,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.025387,0.000000,0.136159,0.000000,0.000000,0.000000
2,0.017853,0.000000,0.000000,0.000000,0.000000,0.000000,0.027372,0.036032,0.000000,0.139139,...,0.002078,0.009899,0.003728,0.026610,0.008374,0.026291,0.000000,0.025024,0.120449,0.031113
3,0.004739,0.000000,0.000000,0.000000,0.000000,0.000000,0.035838,0.042850,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.005945,0.000000,0.000000,0.425126,0.000000,0.050327,0.000000
4,0.000000,0.000000,0.013023,0.000018,0.000000,0.000457,0.000000,0.000000,0.357122,0.024252,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.023475,0.000000,0.000000,0.000000,0.000000


In [18]:
classification_df = pd.DataFrame(df.title.values, columns=["title"])
for i in range(1, n_topics + 1):
    topic = 'main_topic_' + str(i) 
    classification_df.loc[:, topic + '_index'] = t_df.apply(
        lambda row: row.nlargest(i).index.values[i-1] if row.nlargest(i).values[i-1] > 0 else 'none', axis=1)
    classification_df.loc[:, topic + '_words'] = classification_df.apply(
        lambda row: top_words[row[topic + '_index']] if row[topic + '_index'] != 'none' else 'none', axis=1)
classification_df

,title,main_topic_1_index,main_topic_1_words,main_topic_2_index,main_topic_2_words,main_topic_3_index,main_topic_3_words,main_topic_4_index,main_topic_4_words,main_topic_5_index,...,main_topic_21_index,main_topic_21_words,main_topic_22_index,main_topic_22_words,main_topic_23_index,main_topic_23_words,main_topic_24_index,main_topic_24_words,main_topic_25_index,main_topic_25_words
0,CGI_2013,4,"[insurance, health, care, health care, health ...",1,"[tax, deficit, cut, job, spending, pay, busine...",10,"[court, supreme court, supreme, justice, law, ...",15,"[asean, asia, region, china, vietnam, pacific,...",0,...,none,none,none,none,none,none,none,none,none,none
1,Prayer_Breakfast_2016,2,"[god, faith, love, child, family, father, grac...",21,"[muslim, islam, religion, faith, religious, mo...",19,"[africa, ebola, west africa, outbreak, west, d...",11,"[israel, palestinian, israeli, peace, shimon, ...",0,...,none,none,none,none,none,none,none,none,none,none
2,Security_Team_Announcement,9,"[intelligence, terrorist, security, review, al...",23,"[iraq, afghanistan, war, afghan, troop, iraqi,...",10,"[court, supreme court, supreme, justice, law, ...",7,"[war, peace, democracy, human, freedom, africa...",24,...,none,none,none,none,none,none,none,none,none,none
3,Cairo_University,21,"[muslim, islam, religion, faith, religious, mo...",11,"[israel, palestinian, israeli, peace, shimon, ...",23,"[iraq, afghanistan, war, afghan, troop, iraqi,...",7,"[war, peace, democracy, human, freedom, africa...",6,...,none,none,none,none,none,none,none,none,none,none
4,Umpqua_Community_College_Shootings,8,"[gun, background check, gun violence, backgrou...",9,"[intelligence, terrorist, security, review, al...",20,"[boston, mayor, investigation, fbi, local, fed...",2,"[god, faith, love, child, family, father, grac...",10,...,none,none,none,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,ASEAN_Business_2015,15,"[asean, asia, region, china, vietnam, pacific,...",17,"[immigration, immigrant, border, undocumented,...",1,"[tax, deficit, cut, job, spending, pay, busine...",24,"[privacy, cyber, information, company, cyberse...",9,...,none,none,none,none,none,none,none,none,none,none
432,Finance_Crisis_Fee,18,"[financial, crisis, economy, wall street, bank...",none,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
433,YLAI_Town_Hall,12,"[cuban, cuba, castro, havana, relation, embarg...",22,"[tonight, election, hillary, little, vote, cam...",11,"[israel, palestinian, israeli, peace, shimon, ...",19,"[africa, ebola, west africa, outbreak, west, d...",14,...,none,none,none,none,none,none,none,none,none,none
434,Oval_Office_Counter_Terrorism_Agenda,3,"[isil, syria, terrorist, iraq, iraqi, syrian, ...",21,"[muslim, islam, religion, faith, religious, mo...",9,"[intelligence, terrorist, security, review, al...",8,"[gun, background check, gun violence, backgrou...",16,...,none,none,none,none,none,none,none,none,none,none


In [19]:
# get the unique topic id values for the main topic indices
for i in range(1, n_topics + 1):
    topic = 'main_topic_' + str(i) + '_index'
    print(f'main topic {i}')
    print(set(classification_df[topic].values))
    c = list(classification_df[topic].values).count('none')
    print(f"'none' was present {c} times, which is {100*c/len(df):.1f}% of the speeches \n")
    
# main_topic_1 is the only one that never has value 'none', this means that some speeches only have one topic
# main_topic_25 only has 'none', which means that no speeches contain all 25 topics

main topic 1
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}
'none' was present 0 times, which is 0.0% of the speeches 

main topic 2
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 'none'}
'none' was present 6 times, which is 1.4% of the speeches 

main topic 3
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 'none'}
'none' was present 17 times, which is 3.9% of the speeches 

main topic 4
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 'none'}
'none' was present 25 times, which is 5.7% of the speeches 

main topic 5
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 'none'}
'none' was present 44 times, which is 10.1% of the speeches 

main topic 6
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 'none'}
'none' was present 71 times, whi

In [20]:
topics_df = pd.DataFrame(range(n_topics), columns=['topic'])
topics_df['percentage_of_speeches_where_main_topic'] = topics_df.apply(
    lambda row: 100*list(classification_df['main_topic_1_index'].values).count(int(row['topic']))/len(df), axis=1)
topics_df['words'] = topics_df.apply(lambda row: top_words[int(row['topic'])], axis=1)

r = 100/n_topics
print(f'expected percentage per topic: {r:.1f}% \n')
topics_df.sort_values(by='percentage_of_speeches_where_main_topic', ascending=False)

expected percentage per topic: 4.0% 



,topic,percentage_of_speeches_where_main_topic,words
2,2,11.238532,"[god, faith, love, child, family, father, grac..."
22,22,8.944954,"[tonight, election, hillary, little, vote, cam..."
23,23,7.339450,"[iraq, afghanistan, war, afghan, troop, iraqi,..."
1,1,5.733945,"[tax, deficit, cut, job, spending, pay, busine..."
4,4,5.504587,"[insurance, health, care, health care, health ..."
14,14,5.275229,"[school, college, student, education, job, tea..."
15,15,4.816514,"[asean, asia, region, china, vietnam, pacific,..."
3,3,4.357798,"[isil, syria, terrorist, iraq, iraqi, syrian, ..."
7,7,4.357798,"[war, peace, democracy, human, freedom, africa..."
9,9,3.899083,"[intelligence, terrorist, security, review, al..."


In [21]:
topic = 0
classification_df.loc[classification_df['main_topic_1_index']==topic, :]

,title,main_topic_1_index,main_topic_1_words,main_topic_2_index,main_topic_2_words,main_topic_3_index,main_topic_3_words,main_topic_4_index,main_topic_4_words,main_topic_5_index,...,main_topic_21_index,main_topic_21_words,main_topic_22_index,main_topic_22_words,main_topic_23_index,main_topic_23_words,main_topic_24_index,main_topic_24_words,main_topic_25_index,main_topic_25_words
15,UN_Copenhagen_Climate_Change,0,"[energy, oil, clean, climate, clean energy, jo...",9,"[intelligence, terrorist, security, review, al...",15,"[asean, asia, region, china, vietnam, pacific,...",7,"[war, peace, democracy, human, freedom, africa...",13,...,none,none,none,none,none,none,none,none,none,none
76,Global_Climate_Message,0,"[energy, oil, clean, climate, clean energy, jo...",15,"[asean, asia, region, china, vietnam, pacific,...",7,"[war, peace, democracy, human, freedom, africa...",9,"[intelligence, terrorist, security, review, al...",5,...,none,none,none,none,none,none,none,none,none,none
87,BP_Oil_Spill_Presser,0,"[energy, oil, clean, climate, clean energy, jo...",20,"[boston, mayor, investigation, fbi, local, fed...",9,"[intelligence, terrorist, security, review, al...",17,"[immigration, immigrant, border, undocumented,...",10,...,none,none,none,none,none,none,none,none,none,none
98,COP21,0,"[energy, oil, clean, climate, clean energy, jo...",15,"[asean, asia, region, china, vietnam, pacific,...",7,"[war, peace, democracy, human, freedom, africa...",3,"[isil, syria, terrorist, iraq, iraqi, syrian, ...",2,...,none,none,none,none,none,none,none,none,none,none
104,Energy_Security_AAFB,0,"[energy, oil, clean, climate, clean energy, jo...",23,"[iraq, afghanistan, war, afghan, troop, iraqi,...",9,"[intelligence, terrorist, security, review, al...",6,"[iran, nuclear, iranian, deal, nuclear weapon,...",10,...,none,none,none,none,none,none,none,none,none,none
120,Oval_Office_BP,0,"[energy, oil, clean, climate, clean energy, jo...",20,"[boston, mayor, investigation, fbi, local, fed...",2,"[god, faith, love, child, family, father, grac...",23,"[iraq, afghanistan, war, afghan, troop, iraqi,...",21,...,none,none,none,none,none,none,none,none,none,none
122,Energy_Security_Georgetown,0,"[energy, oil, clean, climate, clean energy, jo...",6,"[iran, nuclear, iranian, deal, nuclear weapon,...",8,"[gun, background check, gun violence, backgrou...",none,none,none,...,none,none,none,none,none,none,none,none,none,none
146,Clean_Power_Plan,0,"[energy, oil, clean, climate, clean energy, jo...",4,"[insurance, health, care, health care, health ...",2,"[god, faith, love, child, family, father, grac...",15,"[asean, asia, region, china, vietnam, pacific,...",1,...,none,none,none,none,none,none,none,none,none,none
213,Repeal_Oil_and_Gas_Subsidies,0,"[energy, oil, clean, climate, clean energy, jo...",4,"[insurance, health, care, health care, health ...",none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
221,Paris_Climate_Agreement,0,"[energy, oil, clean, climate, clean energy, jo...",15,"[asean, asia, region, china, vietnam, pacific,...",3,"[isil, syria, terrorist, iraq, iraqi, syrian, ...",24,"[privacy, cyber, information, company, cyberse...",7,...,none,none,none,none,none,none,none,none,none,none


In [26]:
%matplotlib inline
import pyLDAvis.sklearn
import warnings

pyLDAvis.enable_notebook()

vis = pyLDAvis.sklearn.prepare(model, matrix, vectorizer)

pyLDAvis.display(vis)
#warnings.filterwarnings("ignore", category=DeprecationWarning) 
#warnings.filterwarnings("ignore", category=FutureWarning) 

/home/jolienc/.pyenv/versions/3.9.5/envs/mda-project/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/home/jolienc/.pyenv/versions/3.9.5/envs/mda-project/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
